In [ ]:
%load_ext autoreload
%autoreload 2
import logging
import sys
import time
from dataclasses import dataclass
from enum import Enum
from liftout import utils

from autoscript_sdb_microscope_client.structures import AdornedImage, StagePosition, ManipulatorPosition
from liftout.sample import Sample, Lamella, AutoLiftoutState, AutoLiftoutStage

### New Protocol
- Splitting the protocol into separate files. 
    - load into a single settings, dictionary
    - save protocol into individual files 
    - load protocol individually
    - possibly change to loading experiments

In [ ]:
base_path = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout"
import os
from pprint import pprint
from liftout.utils import load_full_config

system_fname = os.path.join(base_path, "config", "system.yaml")
calibration_fname = os.path.join(base_path, "config", "calibration.yaml")
protocol_fname = os.path.join(base_path, "protocol", "protocol.yaml")

settings = load_full_config(
    system_fname, 
    calibration_fname, 
    protocol_fname
)

pprint(settings)




In [ ]:
import matplotlib.pyplot as plt
from autoscript_sdb_microscope_client.structures import AdornedImage
import pandas as pd
from fibsem.detection.utils import load_detection_result, plot_detection_result
from pathlib import Path


# TODO: aggregate multiple datasets together...

path = r"C:\Users\Admin\Github\autoliftout\tools\test\label\data.csv"
base = os.path.dirname(path)
df = pd.read_csv(path)
display(df)

for idx, row in df.iterrows():
    
    det_result = load_detection_result(base, row)
    fig = plot_detection_result(det_result)

    plt.show()
    plt.close()

#### CRossCorrelation Refactor

In [ ]:
from autoscript_sdb_microscope_client.structures import AdornedImage
from fibsem import calibration, acquire
from fibsem import utils as fibsem_utils
import glob
import os
import random
import matplotlib.pyplot as plt
from liftout import utils

from pprint import pprint
path = r"C:\Users\Admin\Github\autoliftout\tools\test\label"
filenames = glob.glob(os.path.join(path, "*.tif"))



In [ ]:
microscope, settings, image_settings = utils.quick_setup()

In [ ]:
eb_image, ib_image = acquire.take_reference_images(microscope, image_settings)


plt.imshow(eb_image.data, cmap="gray")
plt.show()



In [ ]:
eb_image.metadata.scan_settings.scan_size

from liftout import utils
from fibsem.structures import ImageSettings, BeamType


pprint(image_settings)
image_settings = calibration.match_image_settings(ib_image, image_settings)

pprint(image_settings)




### State Testing


In [ ]:
%load_ext autoreload 
%autoreload 2
from liftout import autoliftout
from fibsem import utils as fibsem_utils
from liftout.sample import Sample, Lamella, create_experiment

microscope, settings, image_settings = fibsem_utils.quick_setup()

print(settings.image.save_path)

sample = create_experiment("my_experiment", settings.image.save_path)

for no in range(1, 3):
    lamella = Lamella(sample.path, no)
    sample.positions[lamella._number] = lamella

from pprint import pprint 


In [ ]:
key = list(sample.positions.keys())[0]
lamella = sample.positions[key]
sample = autoliftout.end_of_stage_update(microscope, sample, lamella)

### CrossCorrelation

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import calibration
from autoscript_sdb_microscope_client.structures import AdornedImage

from pprint import pprint


img1 = AdornedImage.load("../test_image.tif")
img2 = AdornedImage.load("../test_image.tif")


import matplotlib.pyplot as plt
plt.imshow(img1.data, cmap="gray")
plt.show()



xcorr = calibration.crosscorrelation(img1.data, img2.data, bp="yes", lp=128, hp=6, sigma=6)
xcorr = calibration.crosscorrelation(img1.data, img2.data, bp=True, lp=128, hp=6, sigma=6)
dx, dy, xcorr = calibration.shift_from_crosscorrelation(img1, img2, lowpass=128, highpass=6, sigma=6, use_rect_mask=True)

plt.imshow(xcorr)
plt.show()

print(dx, dy)


### Edge Detection

In [ ]:
## edge detection

%load_ext autoreload
%autoreload 2

from fibsem.detection import detection
from fibsem import calibration
from autoscript_sdb_microscope_client.structures import AdornedImage
import matplotlib.pyplot as plt

from pprint import pprint

img = AdornedImage.load("../test_image.tif")
centre = img.data.shape[0] // 2, img.data.shape[1] // 2
edge_px, edges_mask = detection.detect_landing_edge(img.data, centre)


fig = plt.figure(figsize=(10, 10))
plt.imshow(edges_mask, cmap="gray")
plt.plot(centre[1], centre[0], "c+", ms=50)
plt.plot(edge_px[1], edge_px[0], "r+", ms=50)
plt.show()

print("centre: ", centre)
print("closest edge: ", edge_px)

In [ ]:
import glob
import random
import os

# NB: problem is bad eucentricty? ib works well, eb not so much
# snap to closest edge is pretty good

filenames = glob.glob("data/**/**/*landing*high_res_*b.tif")
fname = random.choice(filenames)

print(f"{len(filenames)} selected.")
print(f"file: {os.path.basename(fname)}")


for fname in filenames[:2]:
    img = AdornedImage.load(fname)
    centre = img.data.shape[0] // 2, img.data.shape[1] // 2
    edge_px, edges_mask = detection.detect_landing_edge(img.data, centre)

    fig = plt.figure(figsize=(15, 15))
    plt.imshow(edges_mask, cmap="gray")
    plt.plot(centre[1], centre[0], "c+", ms=100, markeredgewidth=2)
    plt.plot(edge_px[1], edge_px[0], "r+", ms=100, markeredgewidth=2)
    plt.plot((centre[1], edge_px[1]), (centre[0], edge_px[0]), "w", ms=100, markeredgewidth=2)
    plt.show()

    print("centre: ", centre)
    print("closest edge: ", edge_px)


In [ ]:
# fig.savefig('test.png',bbox_inches='tight',transparent=True, pad_inches=0)


In [ ]:
# Detection Model

%load_ext autoreload 
%autoreload 2

import glob
import random
import os
from pprint import pprint
from fibsem.detection import detection
from fibsem import calibration
from autoscript_sdb_microscope_client.structures import AdornedImage
import matplotlib.pyplot as plt

from fibsem.detection import utils as det_utils

import numpy as np
from fibsem.detection.detection import DetectionType
from fibsem.detection.DetectionModel import DetectionModel
from PIL import Image

sample = "dm*"
_id = "*e2e6a"  
_id = "*94bb0" 
_id = "*"
img_type = "*"
search = "landing_needle_land_sample"
filenames = glob.glob(f"data/{sample}/{_id}/*{search}*{img_type}.tif") 
random.shuffle(filenames)

print(f"{len(filenames)} selected.")

weights_file = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\model\models\boost_n05_model.pt"
model = DetectionModel(weights_file=weights_file)

N_IMAGES = 25

for fname in filenames[:N_IMAGES]:
    print(f"file: {os.path.basename(fname)}")

    adorned_img = AdornedImage.load(fname)

    det = detection.locate_shift_between_features(
        model, adorned_img, shift_type=(DetectionType.NeedleTip, DetectionType.LandingPost) 
    )

    fig = det_utils.plot_detection_result(det)
    plt.show()
    plt.close()

    # fig = plt.figure(figsize=(15, 15))
    # plt.imshow(detection.edge_detection(adorned_img.data, sigma=3), cmap="gray")
    # plt.show()
    
    # pprint(det.features)


In [ ]:
filenames = glob.glob("data/**/**/*landing_*b.tif")

print(f"{len(filenames)} filenames")
pprint(filenames)

In [ ]:
## Detect Brightness Changes

%load_ext autoreload 
%autoreload 2

import glob
import random
import os
from pprint import pprint
from fibsem.detection import detection
from fibsem import calibration
from autoscript_sdb_microscope_client.structures import AdornedImage
import matplotlib.pyplot as plt

from fibsem.detection import utils as det_utils

import numpy as np
from fibsem.detection.detection import DetectionType
from fibsem.detection.DetectionModel import DetectionModel
from PIL import Image


# Get baseline brightness
# it doesnt need to be a global threshold... brightness should increase when need lands
# just need to look for the increase?

sample = "dm*"
_id = "*e2e6a"  
_id = "*94bb0" 
# _id = "*4f708"
img_type = "lowres_ib"

filenames = glob.glob(f"data/{sample}/{_id}/*needle_liftout*post*{img_type}.tif") + glob.glob(f"data/{sample}/{_id}/*needle_liftout_landed*{img_type}.tif") 
# random.shuffle(filenames)

print(f"{len(filenames)} selected.")

weights_file = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\model\models\boost_n05_model.pt"
model = DetectionModel(weights_file=weights_file)

N_IMAGES = len(filenames)

## INVESTIGATE: 
# does the brightness meaningfully increase when needle is in frame but not landed?
# what are the other factors that influence this?
# measure the needle current?? microscope.state.specimen_current??

landed = []
pre_landing = []

def measure_brightness(img: AdornedImage, crop_size: int = None) -> float:
    cx, cy = img.data.shape[1] //2, img.data.shape[0] // 2

    if crop_size is not None:
        img = img.data[cy-crop_size:cy+crop_size, cx-crop_size:cx+crop_size]
    else:
        img = img.data

    return np.mean(img), img


history = []

for fname in filenames[:N_IMAGES]:
    print(f"file: {os.path.dirname(os.path.dirname(fname))}, {os.path.dirname(fname)[-5:]}, {os.path.basename(fname)}")

    adorned_img = AdornedImage.load(fname)

    # only use the centre of the image???
    # brightness, contrast = np.mean(crop_img), np.std(crop_img)

    brightness, img = measure_brightness(adorned_img, crop_size=250)
    print(f"Brightness: {brightness}") #, contrast: {contrast}")


    if "landed" in fname:
        landed.append(brightness)
    else:
        pre_landing.append(brightness)

    history.append(brightness)

    plt.imshow(img, cmap="gray")
    plt.show()
    
plt.hist(landed, color="blue", alpha=0.5, label="landed", bins=10)
plt.hist(pre_landing, color="red", alpha=0.5, label="pre_landing", bins=10)
plt.legend(loc="best")
plt.show()

print(f"baseline: mean={np.mean(pre_landing)}, std={np.std(pre_landing)}" )
print(f"baseline: mean={np.mean(landed)}, std={np.std(landed)}" )

plt.plot(history, color="blue", label="brightness")
plt.show()


# TODO
# need to get a baseline brightness



In [ ]:
filenames = glob.glob("data/dm*/**/*needle_liftout*ib.tif")

print(f"{len(filenames)} filenames")
pprint(filenames)

In [ ]:
# New liftout strategy

# insert needle

# line up in electron (xy)
#   - should be above the lamella

# loop:
#   move needle down (z)
#   image, check brightness level
#   if meets threshold -> 
#       needle has landed, return
#   continue



In [ ]:
## soft mask lamellla
## cosine stretch

%load_ext autoreload 
%autoreload 2

import glob
import random
import os
from pprint import pprint
from fibsem.detection import detection
from fibsem import calibration
from autoscript_sdb_microscope_client.structures import AdornedImage
import matplotlib.pyplot as plt

from fibsem.detection import utils as det_utils

import numpy as np
from fibsem.detection.detection import DetectionType
from fibsem.detection.DetectionModel import DetectionModel
from fibsem import calibration
from PIL import Image

sample = "dm*"
_id = "*e2e6a"  
_id = "*94bb0" 
_id = "*4f708"
img_type = "*_ib"
search = "ref_trench*"

In [ ]:
# TODO:


In [ ]:
from liftout import utils 

settings = utils.load_full_config()

sample = "dm*"
_id = "*e2e6a"  
_id = "*94bb0" 
_id = "*4f708"
img_type = "*_*b"
search = "*ref_trench*"

weights_file = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\model\models\boost_n05_model.pt"
model = DetectionModel(weights_file=weights_file)


filenames = glob.glob(f"data/{sample}/{_id}/*{search}*{img_type}.tif") 

print(f"{len(filenames)} filenames")

for fname in filenames:

# fname = filenames[0]
    print(f"file: {os.path.dirname(os.path.dirname(fname))}, {os.path.dirname(fname)[-5:]}, {os.path.basename(fname)}")
    img = AdornedImage.load(fname)

    mask = calibration.create_lamella_mask(img, settings)
    # sigma should change based on size of image (12ish)

    # mask out reference image...

    # mask = create_vertical_mask(img)
    print(np.mean(img.data), np.std(img.data))
    norm_img = (img.data - np.mean(img.data)) / np.std(img.data)
    print(np.min(norm_img), np.max(norm_img))
    # masked_img = AdornedImage(data=np.asarray(norm_img * mask, dtype=np.uint8), metadata=img.metadata)
    masked_img = np.asarray(norm_img * mask)

    # fig, ax = plt.subplots(1, 2, figsize=(15, 15))
    # ax[0].imshow(img.data, cmap="gray")
    # ax[1].imshow(masked_img.data, cmap="gray")
    # plt.show()

    # det = detection.locate_shift_between_features(
    #     model, masked_img, shift_type=(DetectionType.ImageCentre, DetectionType.LamellaCentre)
    # )

    fig, ax = plt.subplots(1, 2, figsize=(15, 15))
    ax[0].imshow(img.data, cmap="gray")
    ax[1].imshow(masked_img.data, cmap="gray")
    # ax[2].imshow(det.display_image)
    plt.show()

In [ ]:

# reference    
"ref_trench_low_res_eb"
"ref_trench_high_res_eb"
"ref_trench_low_res_ib"
"ref_trench_high_res_ib"


In [ ]:
# land lamella with edge detection

%load_ext autoreload 
%autoreload 2

import glob
import random
import os
from pprint import pprint
from fibsem.detection import detection
from fibsem import calibration
from autoscript_sdb_microscope_client.structures import AdornedImage
import matplotlib.pyplot as plt

from fibsem.detection import utils as det_utils

import numpy as np
from fibsem.detection.detection import DetectionType
from fibsem.detection.DetectionModel import DetectionModel
from PIL import Image
from fibsem.structures import Point
from skimage import filters


sample = "dm*"
_id = "*e2e6a"  
_id = "*94bb0" 
_id = "*"
img_type = "*"
search = "sharpen*"
filenames = glob.glob(f"data/{sample}/{_id}/*{search}*{img_type}.tif") 
random.shuffle(filenames)

print(f"{len(filenames)} selected.")

weights_file = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\model\models\boost_n05_model.pt"
model = DetectionModel(weights_file=weights_file)

N_IMAGES = 5

for fname in filenames[:N_IMAGES]:
    print(f"file: {os.path.basename(fname)}")

    img = AdornedImage.load(fname)

    # det = detection.locate_shift_between_features(
    #     model, adorned_img, shift_type=(DetectionType.NeedleTip, DetectionType.LandingPost) 
    # )

    # fig = det_utils.plot_detection_result(det)
    # plt.show()
    # plt.close()

    mask = np.zeros_like(img.data)
    cy, cx = mask.shape[0] // 2, mask.shape[1] // 3
    mask[cy//3:, 0:cx] = 1
    # mask[cy-100:cy+250:, 0:cx] = 1
    
    masked_img = AdornedImage(data=np.asarray(img.data * mask, dtype=np.uint8), metadata=img.metadata)


    edge_img = detection.edge_detection(filters.gaussian(masked_img.data, sigma=3), sigma=3)

    start_px = Point(x=cx//2, y=cy)

    end_px, edge_img = detection.detect_closest_edge(masked_img.data, (start_px.y, start_px.x))

    # detect closest edge?

    # detect right edge?

    print(np.unique(edge_img))


    # detection.

    edges_x = filters.sobel_v(filters.gaussian(img.data, sigma=6))
    edges_y = filters.sobel_h(filters.gaussian(img.data, sigma=6))

    fig, ax = plt.subplots(1, 4, figsize=(25, 20))

    ax[0].imshow(img.data, cmap="gray")
    ax[1].imshow(edges_x, cmap="gray")
    ax[2].imshow(edges_y, cmap="gray")

    ax[3].imshow(edge_img, cmap="turbo")

    # ax[3].imshow(masked_img.data, cmap="gray")
    ax[3].plot(start_px.x, start_px.y, "r+", ms=50, markeredgewidth=2)
    ax[3].plot(end_px.x, end_px.y, "c+", ms=50, markeredgewidth=2)
    
    # ax[3].plot([0, cx], [cy-100, cy-100], "w--", ms=50, markeredgewidth=2)
    # ax[3].plot([0, cx], [cy+100, cy+100], "w--", ms=50, markeredgewidth=2)

    # ax[2].imshow(img.data, cmap="gray")
    plt.show()
    
    # pprint(det.features)


In [ ]:
## perspective transform / cosine stretch

%load_ext autoreload 
%autoreload 2

from fileinput import filename
import glob
import random
import os
from pprint import pprint

from fibsem.detection import detection
from fibsem import calibration
from autoscript_sdb_microscope_client.structures import AdornedImage
import matplotlib.pyplot as plt

from fibsem.detection import utils as det_utils

import numpy as np
from fibsem.detection.detection import DetectionType
from fibsem.detection.DetectionModel import DetectionModel
from PIL import Image
from fibsem.structures import Point
from skimage import filters

from liftout import utils
utils.configure_logging(r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\tools\auto\test_sample2_2022-07-22.04-31-54PM")
settings = utils.load_full_config()

_id = "*b0"
_id = "*6a" 
_id = "*af" 
_id = "*08"

ids = ["6a4e5fad-5484-4f20-8d9f-53d111394bb0","552a8bea-0a8a-4f97-80fb-0fe66a5e2e6a", 
"9ebe8ed1-1ef0-4f67-a62b-bbf2abb4d2af","73d5fae8-9df6-4cc8-91e3-e2da1b94f708"]

# cross correlate
def crosscorrelate_and_plot(ref_image, new_image, rotate: bool = False):

    # rotate ref
    if rotate:
        ref_image = calibration.rotate_image(ref_image)

    dx, dy, xcorr = calibration.shift_from_crosscorrelation(
        ref_image, new_image, lowpass=128, use_rect_mask=True
    )

    pixelsize = ref_image.metadata.binary_result.pixel_size.x
    dx_p, dy_p = int(dx / pixelsize), int(dy / pixelsize)

    print(dx, dy)
    print(dx_p, dy_p)

    shift = np.roll(new_image.data, (-dy_p, -dx_p), axis=(0, 1))

    fig, ax = plt.subplots(1, 4, figsize=(30, 30))
    ax[0].imshow(ref_image.data, cmap="gray")
    ax[0].plot(shift.shape[1]//2, shift.shape[0]//2, "r+", ms=50, markeredgewidth=2)
    ax[0].set_title(f"Reference (rotate={rotate})")
    ax[1].imshow(new_image.data, cmap="gray")
    ax[1].plot(shift.shape[1]//2, shift.shape[0]//2, "r+", ms=50, markeredgewidth=2)
    ax[1].set_title(f"New Image")
    ax[2].imshow(xcorr, cmap="turbo")
    ax[2].plot(shift.shape[1]//2, shift.shape[0]//2, "r+", ms=50, markeredgewidth=2)
    ax[2].set_title("XCORR")
    ax[3].imshow(shift, cmap="gray")
    ax[3].plot(shift.shape[1]//2, shift.shape[0]//2, "r+", ms=50, markeredgewidth=2)
    ax[3].set_title("New Image Shifted")
    plt.show()


def mask_adorned_image(img: AdornedImage, mask) -> AdornedImage:
    masked_img = AdornedImage(data=np.asarray(img.data * mask, dtype=np.uint8), metadata=img.metadata)

    return masked_img

path = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\tools\auto\data\dm-embryo-3_20220719.104850"

for _id in ids:

    ref_fname = [os.path.join(path, _id, "ref_trench_low_res_ib.tif"), os.path.join(path, _id, "ref_trench_high_res_ib.tif")]#, os.path.join(path, _id, "jcut_highres_ib.tif")]
    new_fname = [os.path.join(path, _id, "ref_trench_low_res_eb.tif"), os.path.join(path, _id, "ref_trench_high_res_eb.tif")]#, os.path.join(path, _id, "jcut_highres_eb.tif")]
    # new_fname = [glob.glob(os.path.join(path, _id, "drift_correction_ML_final_*_eb.tif"))[0], 
    #     glob.glob(os.path.join(path, _id, "drift_correction_*_eb.tif"))[0]]

    use_mask = False
    use_vertical_mask = False

    for idx in range(len(ref_fname)):
        ref_image = AdornedImage.load(ref_fname[idx])
        new_image = AdornedImage.load(new_fname[idx])

        ref_image = AdornedImage(data=np.roll(ref_image.data, shift=(75, 75), axis=(0, 1)), metadata=ref_image.metadata)

        if use_mask:
            mask = calibration.create_lamella_mask(ref_image, settings, factor=2)
            ref_image = mask_adorned_image(ref_image, mask)
            new_image = mask_adorned_image(new_image, mask)
        
        if use_vertical_mask:
            mask = calibration.create_vertical_mask(ref_image, w=400, h=400)
            ref_image = mask_adorned_image(ref_image, mask)
            new_image = mask_adorned_image(new_image, mask)
        

        crosscorrelate_and_plot(ref_image, new_image, rotate=False)


In [ ]:
## Focus Band Alignment


# land lamella with edge detection

%load_ext autoreload 
%autoreload 2

import glob
import random
import os
from pprint import pprint
from fibsem.detection import detection
from fibsem import calibration
from autoscript_sdb_microscope_client.structures import AdornedImage
import matplotlib.pyplot as plt

from fibsem.detection import utils as det_utils

import numpy as np
from PIL import Image
from fibsem.structures import Point
from skimage import filters

import skimage

from skimage.morphology import disk
from skimage.filters.rank import gradient




sample = "dm*"
_id = "*e2e6a"  
_id = "*94bb0" 
_id = "*"
img_type = "*_*b"
search = "ref_landing*"
filenames = glob.glob(f"data/{sample}/{_id}/*{search}*{img_type}.tif") 
random.shuffle(filenames)

print(f"{len(filenames)} selected.")

def calculate_maximum_focus_band(img, 
        method = "accutance", steps=16):

    sy, sx = img.data.shape
    
    step = sy // steps   
    start_idx = range(0, sy, step)
    
    bands = np.zeros_like(img.data)
    sharpness_metrics = {}
    for idx in start_idx:
        
        crop = img.data[idx:idx+step, :]
        mask = np.zeros_like(img.data)
        mask[idx:idx+step, :] = crop


        # sharpness (Acutance: https://en.wikipedia.org/wiki/Acutance
        if method == "accutance":
            out = gradient(skimage.filters.median(crop), disk(5))
        if method == "grad_x":
            out = filters.sobel_v(filters.gaussian(crop, sigma=3))
        if method == "grad_y":
            out = filters.sobel_h(filters.gaussian(crop, sigma=3))

        sharpness = np.mean(out)

        # print(idx, step, sharpness)
        sharpness_metrics[idx] = sharpness
        bands[idx:idx+step, :] = sharpness

    sorted_sharpness = sorted(sharpness_metrics.items(), key=lambda x: x[1], reverse=True)
    print("-------")
    pprint(sorted_sharpness)
    print("--------")    

    # distance from centre (y)
    max_idx = np.clip(sorted_sharpness[0][0] +(step // 2), 0, sy) # centre of band
    
    cy, cx = img.data.shape[0] // 2, img.data.shape[1] // 2
    pixelsize = img.metadata.binary_result.pixel_size.x

    offset_px = cy-max_idx
    offset_metres = offset_px * pixelsize

    print(f"Centre: {cy}, Maximum: {max_idx} sharpness: {sorted_sharpness[0][1]}")
    print(f"Difference: {offset_px}px, {offset_metres}m")

    return offset_px, offset_metres, max_idx, bands

N_IMAGES = 5

for fname in filenames[:N_IMAGES]:
    print(f"file: {os.path.basename(fname)}")

    img = AdornedImage.load(fname)

    offset_px, offset_metres, max_idx, bands = calculate_maximum_focus_band(img, method="accutance", steps=16)

    fig, ax = plt.subplots(1, 2, figsize=(25, 25))
    ax[0].imshow(img.data, cmap="gray")
    ax[0].plot([0, img.data.shape[1]-1], [max_idx-1, max_idx-1], "r--", ms=50, markeredgewidth=2)
    ax[1].imshow(bands, cmap="Blues_r")

    accutance = gradient(skimage.filters.median(np.copy(img.data)), disk(5))
    edges_x = filters.sobel_v(filters.gaussian(img.data, sigma=3))
    edges_y = filters.sobel_h(filters.gaussian(img.data, sigma=3))
    

    fig, ax = plt.subplots(1, 3, figsize=(25, 25))
    ax[0].imshow(accutance, cmap="gray")
    ax[1].imshow(edges_x, cmap="gray")
    ax[2].imshow(edges_y, cmap="gray")

    plt.show()

    # move z